# Лекция Python 1
## Один из реалистичных примеров ООП
### Объектно-ориентированное программирование и наследование: отношения "является"

In [4]:
from __future__ import print_function

class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
    
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)

    def work(self):
        print(self.name, "does stuff") # типо что-то делает

    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)

class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)

    def work(self):
        print(self.name, "makes food") # готовит еду

class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)
    
    def work(self):
        print(self.name, "interfaces with customer") # Взаимодействует с клиентом

class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)   

    def work(self):
        print(self.name, "makes pizza") # готовит пиццу

if __name__ == "__main__" :
    bob = PizzaRobot ('bob')            # Создать робота по имени bob
    print(bob)                          # Выполняется унаследованный метод__repr__
    bob.work()                          # Выполняется действие, специфичное для типа
    bob.giveRaise (0.20)                # Повысить зарплату роботу bob на 20%
    print(bob); print()
    
for klass in Employee, Chef, Server, PizzaRobot: 
    obj = klass(klass.__name__)
    obj.work()

<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>

Employee does stuff
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


### Объектно-ориентированное программирование и наследование: отношения "имеет"

In [5]:
# Файл pizzashop.ру (Python 2.Х + З.Х)
from __future__ import print_function 

class Customer:
    def __init__(self, name):
        self.name = name
    
    def order(self, server):
        print(self.name, "orders from", server) # заказы от
    
    def pay(self, server):
        print(self.name, "pays for item to", server) # плата за единицу
        

class Oven:
    def bake(self): 
        print("oven bakes") # духовой шкаф выпекает


class PizzaShop:
    def __init__(self) :
        self.server = Server('Pat') # Внедрить другие объекты
        self.chef = PizzaRobot('Bob')  # Робот по имени bob
        self.oven = Oven()

    def order(self, name):
        customer = Customer(name) # Активизировать другие объекты
        customer.order(self.server) # Заказы клиента, принятые официантом
        self.chef.work() 
        self.oven.bake() 
        customer.pay(self.server)

if __name__ == "__main__": 
    scene = PizzaShop() # Создать составной объект
    scene.order('Homer') # Эмулировать заказ клиента Homer
    print('...')
    scene.order('Shaggy') # Эмулировать заказ клиента Shaggy

Homer orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>


### Обработчики потоков данных

В качестве примера композиции, который может оказаться чуть более осязаемым, чем роботы по приготовлению пиццы, напишем обобщенную функцию обработчика потоков данных:

In [ ]:
def processor(reader, converter, writer): 
    while True:
        data = reader.read() 
        if not data: 
            break 
        data = converter(data) 
        writer.write(data)


Вместо применения простой функции мы можем написать код класса, который для выполнения своей работы использует композицию, чтобы обеспечить большую структурированность и поддержку наследования. 


In [1]:
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader 
        self.writer = writer

    def process(self): 
        while True:
            data = self.reader.readline() 
            if not data: 
                break
            data = self.converter(data) 
            self.writer.write(data)

    def converter(self, data):
        assert False,  'converter must be defined' # Или сгенерировать исключение

In [2]:
class Uppercase(Processor):
    def converter(self, data):
        return data.upper ()


if __name__ == '__main__' : 
    import sys
    obj = Uppercase(open('./trispam.txt'), sys.stdout) 
    obj.process()

SPAM
SPAM
SPAM! 


In [3]:
class HTMLize:
    def write(self, line):
        print('<PRE>%s</PRE>' % line.rstrip())

Uppercase(open('trispam.txt'), HTMLize()).process()

<PRE>SPAM</PRE>
<PRE>SPAM</PRE>
<PRE>SPAM!</PRE>


In [ ]:
import pickle